# Expressive monkeys

- https://adventofcode.com/2022/day/11

The monkey business here mostly involves parsing some text into something we can execute as Python.

Luckily the operations can be inverted easily to put the right-hand operand on the left (`old * value` is the same as `value * old`), so we can use [`functools.partial()`](https://docs.python.org/3/library/functools.html#functools.partial) together with [`operator` functions](https://docs.python.org/3/library/operator.html) to create callables to handle these. The exception is the `old * old` operation; that's the same as raising `old` to the power 2, however, and because the [`pow()` function](https://docs.python.org/3/library/functions.html#pow) accepts keyword arguments we can translate `old * old` to `partial(pow, exp=2)`. And just in case `old + old` is a possible expression, the code maps that option to `2 * old`.


In [1]:
import operator
import re
from dataclasses import dataclass
from functools import partial, reduce
from heapq import nlargest
from typing import Callable, Final, Iterator, Self, TypeAlias

MonkeyNum: TypeAlias = int
WorryLevel: TypeAlias = int


_parse: Final[re.Pattern[str]] = re.compile(
    r"\s*Starting items: (?P<items>[\d, ]*)\n"
    r"\s*Operation: new = old (?P<op>[+*]) (?P<operand>\d+|old)\n"
    r"\s*Test: divisible by (?P<divisible_by>\d+)\n"
    r"\s*If true: throw to monkey (?P<if_true>\d)\n"
    r"\s*If false: throw to monkey (?P<if_false>\d)"
)


@dataclass
class Monkey:
    items: list[WorryLevel]
    operation: Callable[[WorryLevel], WorryLevel]
    divisible_by: int
    if_true: MonkeyNum
    if_false: MonkeyNum
    relief: int = 3

    def __iter__(self) -> Iterator[tuple[MonkeyNum, WorryLevel]]:
        """The items that the monkey throws to the other monkeys"""
        op, relief = self.operation, self.relief
        if_true, divisible_by, if_false = self.if_true, self.divisible_by, self.if_false
        items, self.items = self.items, []
        for old in items:
            new = op(old) // relief
            yield (if_false if new % divisible_by else if_true), new

    def __len__(self) -> int:
        """Number of items this monkey inspects"""
        return len(self.items)

    @classmethod
    def from_text(cls, text: str, relief: int = 3) -> Self:
        """Create a monkey from puzzle input"""
        match = _parse.search(text)
        assert match is not None
        items = [int(item) for item in match["items"].split(",")]
        divisible_by, if_true, if_false = map(
            int, match.group("divisible_by", "if_true", "if_false")
        )
        match (match["op"], match["operand"]):
            case ("+", "old"):
                operation = partial(operator.mul, 2)  # old + old => 2 * old
            case ("*", "old"):
                operation = partial(pow, exp=2)  # old * old => old ** 2
            case ("+", operand):
                operation = partial(
                    operator.add, int(operand)
                )  # old + <num> => <num> + old
            case ("*", operand):
                operation = partial(
                    operator.mul, int(operand)
                )  # old * <num> => <num> * old
            case (op, operand):
                raise ValueError(f"Invalid monkey operation old {op} {operand}")
        return cls(items, operation, divisible_by, if_true, if_false, relief)


def watch_stuff_slinging_simian_shenanigans(
    monkeys: list[Monkey], rounds: int = 20
) -> int:
    """Watch the monkeys throw their items to each other

    Returns the resulting monkey business level.

    """
    inspected = [0] * len(monkeys)
    lcm = reduce(operator.mul, (monkey.divisible_by for monkey in monkeys))
    for _ in range(rounds):
        for num, monkey in enumerate(monkeys):
            for to_, item in monkey:
                inspected[num] += 1
                monkeys[to_].items.append(item % lcm)
    return operator.mul(*nlargest(2, inspected))


example = """\
Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1
""".split("\n\n")
example_monkeys = [Monkey.from_text(monkey) for monkey in example]

assert watch_stuff_slinging_simian_shenanigans(example_monkeys) == 10605

In [2]:
import aocd

descriptions = aocd.get_data(day=11, year=2022).split("\n\n")
monkeys = [Monkey.from_text(monkey) for monkey in descriptions]
print("Part 1:", watch_stuff_slinging_simian_shenanigans(monkeys))

Part 1: 108240


# Part 2, huge worry levels

The puzzle description hints at what removing the division by 3 will do to the worry levels! Python integers may have no theoretical limits to the number of digits, but your available memory is eaten rapidly by the ~3 dozen massive numbers.

Luckily, we can cap the worry levels to the product of all the monkeys' divisible values; e.g. the example monkeys have divisible test values 17, 13, 19 and 23, so worry levels can be reduced by taking their modulo with 96577; any value that's divisible by 96577 is also divisible by 17, 13, 19, and 23. Because the puzzle always uses _prime numbers_ for the divisible tests, multiplying them together gives us the [_least common multiple_](https://en.wikipedia.org/wiki/Least_common_multiple) (or `lcm`) in our monkey throwing ring. If they were _not_ prime numbers, we'd have to do more work but the number could then be smaller still.

I calculate the `lcm` in the `watch_stuff_slinging_simian_shenanigans()` function and apply it to all the items being flung.


In [3]:
example_monkeys = [Monkey.from_text(monkey, 1) for monkey in example]
assert watch_stuff_slinging_simian_shenanigans(example_monkeys, 10000) == 2713310158

In [4]:
monkeys = [Monkey.from_text(monkey, 1) for monkey in descriptions]
print("Part 1:", watch_stuff_slinging_simian_shenanigans(monkeys, 10000))

Part 1: 25712998901
